
Voy a escribir un programa que calcule las  ́orbitas de los planetas a partir de unas condiciones iniciales y voy a ver si se cumple la tercera ley de Kepler $\frac{T2}{a^3}=cte$, y el Teorema del virial que relaciona la energıa cinetica y potencial:$〈T〉=−\frac{〈V〉}{2} $ 

In [ ]:
from numpy import sin,cos,array,linspace,pi,sqrt,copy,shape,arctan,amin,amax
from matplotlib.pyplot import plot,show,xlabel,ylabel,scatter,title,legend,figure,xlim,ylim

#valores de los planetas
valores=array([['Mercurio',2.4e23,0.39,0.206],['Venus',4.9e24,0.72,0.007],['Tierra',6e24,1,0.017],['Marte',6.6e23,1.52,0.093],['Júpiter',1.9e27,5.2,0.048],['Saturno',5.7e26,9.54,0.056],['Urano',8.8e25,19.19,0.046],['Neptuno',1.03e26,30.06,0.01],['Plutón',1.3e22,39.53,0.248]])
filas, columnas = shape(valores)

# constantes del sistema
GM=4*pi**2

# definimos nuestro intervalo y el paso
h=1e-4
delta=1e-14 #cuanto menor sea mas pasos va a dar (más pequeños) y mas precisión

# definimos nuestra función vectorial, vamos a diferenciar entre f1 y f2 para poder trabajar con el Runge-Kutta de paso adaptado
#en f cogemos los valores de r y en f2 los cogemos de r2
def f(r): 
    x=r[0] #los pongo pero la x y la y no sirven porque no estan en la integral 
    y=r[1]
    xvel=r[2]
    yvel=r[3]

    fx=xvel
    fy=yvel
    erre=sqrt(x**2+y**2)

    fxvel=-GM*x/erre**3
    fyvel=-GM*y/erre**3

    return array([fx,fy,fxvel,fyvel],float)

def f2(r2): 
    x=r2[0] #los pongo pero la x y la y no sirven porque no estan en la integral 
    y=r2[1]
    xvel=r2[2]
    yvel=r2[3]

    fx=xvel
    fy=yvel
    erre=sqrt(x**2+y**2)
    fxvel=-GM*x/erre**3
    fyvel=-GM*y/erre**3

    return array([fx,fy,fxvel,fyvel],float)

# inicializamos nuestras arrays


for i in range(filas):
    #vamos a ir metiendo los valores al integrar en los siguientes vectores
    xp=[]
    yp=[]
    xvelp=[]
    yvelp=[]

    t=0 #partimos de t=0 y vamos a ir sumando el tiempor por cada paso
    vueltas=0 #gracias a q vamos a contar el número de vueltas vamos a poder parar el código en la primera vuelta y q no corra infinitamente
    T=0 #para sacar la energia cinetica en cada paso
    V=0 #para sacar la energia potencial en cada caso

    # fijamos las condiciones iniciales
    v1=sqrt(GM*(1-float(valores[i][3]))/((float(valores[i][2])*(1+float(valores[i][3])))))

    r=array([float(valores[i][2])*float(valores[i][3])+float(valores[i][2]),0,0,v1],float)#valor incial de x y la vel
    r2=array([float(valores[i][2])*float(valores[i][3])+float(valores[i][2]),0,0,v1],float)#este lo uso para hacer los de 2h

    #me meto ya los primeros valores para luego graficar
    xp.append(r2[0])
    yp.append(r2[1])
    xvelp.append(r2[2])
    yvelp.append(r2[3])

    print(valores[i][0]) #imprimimos el nombre del planeta
    Einic=((1.5e11/3.154e7)**2)*(0.5*float(valores[i][1])*v1**2-GM*float(valores[i][1])/(float(valores[i][2])+float(valores[i][2])*float(valores[i][3])))
    print('E inicial:', Einic,"[J]")#calculamos la energía total incial, se multiplica por 231.822 para pasar a julios
    

    while True:     
        r0=copy(r) #esto me va a servir para cuando se de que rho<1, para tener guardado el r antes de las iteraciones y volver a ese mismo valor

        for j in range(2): #hago h y luego h
            k1=h*f(r)
            k2=h*f(r+0.5*k1)
            k3=h*f(r+0.5*k2)
            k4=h*f(r+k3)

            r+=(k1+2*k2+2*k3+k4)/6 #voy a conseguir el valor despues de hacer dos veces h
    
        #ahora directamente usamos paso=2h
        k1_2=2*h*f2(r2)
        k2_2=2*h*f2(r2+0.5*k1_2)
        k3_2=2*h*f2(r2+0.5*k2_2)
        k4_2=2*h*f2(r2+k3_2)
    
        r2+=(k1_2+2*k2_2+2*k3_2+k4_2)/6

        rho=30*h*delta/sqrt(((r[0]-r2[0])/30)**2+((r[1]-r2[1])/30)**2) #condición de convergencia

        if rho>1: #utilizamos el 2h (nuestra precisión es mayor que el objetivo) en este caso avanzamos
            #introducimos los valores en nuestros vectores
            xp.append(r2[0]) 
            yp.append(r2[1])
            xvelp.append(r2[2])
            yvelp.append(r2[3])

            t+=2*h #sumamos el tiempo del paso 2h
            #Calculamos V y T medio en ese intervalo de tiempo y lo multiplicamos por el paso que hemos dado (luego dividiremos entre el total del tiempo dando el promedio de ambos)
            #de momento no pasamos a unidades de julios, pasamos en el último paso
            V+=2*h*GM*float(valores[i][1])/(0.5*(sqrt(xp[-1]**2+yp[-1]**2)+sqrt(xp[-2]**2+yp[-2]**2))) #coge el r medio entre los valores para tener menos error 
            T+=2*h*0.5*float(valores[i][1])*(0.5*(sqrt(xvelp[-1]**2+yvelp[-1]**2)+sqrt(xvelp[-2]**2+yvelp[-2]**2)))**2 #igual q antes, cojo v medio

            h=h*rho**(1/4)
            r=copy(r2) #le decimos que r es los valores de r2 para que empiece en la prox iteración desde ahí 
            
            if yp[-1]>0 and yp[-2]<0: #condición de acabar vuelta (y pasa de negativa a positiva)
                vueltas+=1
                if vueltas>1: #tras una vuelta calculamos E final y sabiendo ya el periodo de la órbita (el t al que hemos ido sumando 2*h) podemos sacar el promedio de las energías
                    Efin=((1.5e11/3.154e7)**2)*(0.5*float(valores[i][1])*(xvelp[-1]**2+yvelp[-1]**2)-GM*float(valores[i][1])/sqrt(xp[-1]**2+yp[-1]**2))
                    print('E final:',Efin,"[J]")
                    print("Diferencia de energías es",(Efin-Einic)*100/Einic,"%" )
                    Tprom=((1.5e11/3.154e7)**2)*T/t
                    Vprom=((1.5e11/3.154e7)**2)*V/(2*t)
                    print('<T>=',Tprom,"[J]")
                    print('-<V>/2=',Vprom,"[J]")
                    print("Diferencia de energías promeido es",abs(Tprom-Vprom)*100/Tprom,"%" )
                    break

        else: #en este caso volvemos al punto de partida con otra h más pequeña
            h=h*rho**(1/4)
            r=copy(r0)
            r2=copy(r0)


    figure(figsize=(5,5))   
    print('Constante de Kepler:',(t/vueltas)**2/(float(valores[i][2]))**3)
    scatter(0,0,c="yellow",label="Sol")
    scatter(xp,yp,s=0.1)
    
    #estas cuatro lineas las hemos hecho para que el eje de las x tenga el mismo rango de medidas q el de las y, en consecuencia ver la imagen tal y como se da en la realidad (sin deformaciones debidas al rango de los ejes)
    limmin=amin([amin(xp),amin(yp)])
    limmas=amax([amax(xp),amax(yp)])
    xlim(limmin+0.1*limmin,limmas+0.1*limmas)
    ylim(limmin+0.1*limmin,limmas+0.1*limmas)

    title(valores[i][0])
    legend()
    xlabel("x")
    ylabel("y")
    print()
    show()

Podemos ver como para todos nuestros planetas el valor de la energía total inicial y final se conserva (el error es muy bajo), se cumple el Teorema del virial y la constante de Kepler es muy cercana a 1. La órbita es cerrada tal y como se puede apreciar en  las gráficas

Ahora voy a modificar la fuerza de Newton incluyendo la primera correccion de la Relatividad General(para Mercurio). Esta correccion deberia hacer que la  ́orbita no se cierre, y por tanto, el perihelio avanza con cada vuelta. El ritmo de avance de este angulo  ́$θ′(t)$ puede determinarse calculando lo q ue avanza por cada vuelta el perihelio. A partir de esto se puede extrapolar el valor real $('43′′/siglo)$

In [ ]:
from numpy import sin,cos,array,linspace,pi,sqrt,copy,shape,arctan,amin,amax,arctan,mean,argmin
from matplotlib.pyplot import plot,show,xlabel,ylabel,scatter,title,legend,figure,xlim,ylim

#valores de los planetas

valores=array([['Mercurio',2.4e23,0.39,0.206],['Venus',4.9e24,0.72,0.007],['Tierra',6e24,1,0.017],['Marte',6.6e23,1.52,0.093],['Júpiter',1.9e27,5.2,0.048],['Saturno',5.7e26,9.54,0.056],['Urano',8.8e25,19.19,0.046],['Neptuno',1.03e26,30.06,0.01],['Plutón',1.3e22,39.53,0.248]])
filas, columnas = shape(valores)

# constantes del sistema
GM=4*pi**2
c=600
# definimos nuestro intervalo y el paso
h=1e-5
delta=0.5e-14 #cuanto menor sea mas pasos va a dar (más pequeños) y mas precisión

# definimos nuestra función vectorial, vamos a diferenciar entre f1 y f2 para poder trabajar con el Runge-Kutta de paso adaptado
#en f cogemos los valores de r y en f2 los cogemos de r2
def f(r): 
    x=r[0] #los pongo pero la x y la y no sirven porque no estan en la integral 
    y=r[1]
    xvel=r[2]
    yvel=r[3]

    fx=xvel
    fy=yvel
    erre=sqrt(x**2+y**2)
    v=sqrt(xvel**2+yvel**2)

    fxvel=-GM*((1-4*GM/(erre*c**2)+v**2/(c**2))*x-4*(xvel*x+y*yvel)*xvel/(c**2))/(erre**3)
    fyvel=-GM*((1-4*GM/(erre*c**2)+v**2/(c**2))*y-4*(xvel*x+y*yvel)*yvel/(c**2))/(erre**3)
    return array([fx,fy,fxvel,fyvel],float)

def f2(r2): 
    x=r2[0] #los pongo pero la x y la y no sirven porque no estan en la integral 
    y=r2[1]
    xvel=r2[2]
    yvel=r2[3]

    fx=xvel
    fy=yvel
    erre=sqrt(x**2+y**2)
    v=sqrt(xvel**2+yvel**2)

    fxvel=-GM*((1-4*GM/(erre*c**2)+v**2/(c**2))*x-4*(xvel*x+y*yvel)*xvel/(c**2))/(erre**3)
    fyvel=-GM*((1-4*GM/(erre*c**2)+v**2/(c**2))*y-4*(xvel*x+y*yvel)*yvel/(c**2))/(erre**3)

    return array([fx,fy,fxvel,fyvel],float)

# inicializamos nuestras arrays


for i in range(1):
    #vamos a ir metiendo los valores al integrar en los siguientes vectores
    xp=[]
    yp=[]
    rp=[]
    xvelp=[]
    yvelp=[]

    figure(figsize=(25,25))

    semiejemayor=float(valores[i][2])*float(valores[i][3])+float(valores[i][2])
    x0=[] #voy a ir metiendo los valores en los q la órbita está en el semieje menor
    y0=[] # lo mismo pero con i
    t0=[0] #voy a ir metiendo los valores de tiempo para cada paso
    t=0 #aqui me voy a ir poniendo los tiempos
    vueltas=0

    # fijamos las condiciones iniciales
    v1=sqrt(GM*(1-float(valores[i][3]))/((float(valores[i][2])*(1+float(valores[i][3])))))

    r=array([semiejemayor,0,0,v1],float)#valor incial de x y la vel
    r2=array([semiejemayor,0,0,v1],float)#este lo uso para hacer los de 2h

    #me meto ya los primeros valores para luego graficar
    xp.append(r2[0])
    yp.append(r2[1])
    xvelp.append(r2[2])
    yvelp.append(r2[3])

    while True:     
        r0=copy(r) #esto me va a servir para cuando se de que rho<1, para tener guardado el r antes de las iteraciones y volver a ese mismo valor

        for j in range(2): #hago h y luego h
            k1=h*f(r)
            k2=h*f(r+0.5*k1)
            k3=h*f(r+0.5*k2)
            k4=h*f(r+k3)

            r+=(k1+2*k2+2*k3+k4)/6 #voy a conseguir el valor despues de hacer dos veces h
    
        #ahora directamente usamos paso=2h
        k1_2=2*h*f2(r2)
        k2_2=2*h*f2(r2+0.5*k1_2)
        k3_2=2*h*f2(r2+0.5*k2_2)
        k4_2=2*h*f2(r2+k3_2)
    
        r2+=(k1_2+2*k2_2+2*k3_2+k4_2)/6

        rho=30*h*delta/sqrt(((r[0]-r2[0])/30)**2+((r[1]-r2[1])/30)**2)

        if rho>1: #utilizamos el 2h (nuestra precisión es mayor que el objetivo) en este caso avanzamos
            #introducimos los valores en nuestros vectores
            xp.append(r2[0])
            yp.append(r2[1])
            rp.append(r2[0]**2+r2[1]**2)
            xvelp.append(r2[2])
            yvelp.append(r2[3])
        
            t+=2*h #sumamos el tiempo del paso 2h

            h=h*rho**(1/4)
            r=copy(r2) #le decimos que r es los valores de r2 para que empiece en la prox iteración desde ahí
            
            if yp[-1]>0 and yp[-2]<0: #condición de acabar vuelta (y pasa de negativa a positiva)
                scatter(xp,yp,s=0.1) #me dibujo la vuelta dada 
                vueltas+=1
                pos=argmin(rp) #me quedo con la posicion para la q el radio es menor
                x0.append(xp[pos]) #cojo la x y la y de esa posición
                y0.append(yp[pos])
                t0.append(t) #me meto el tiempo en el q se día ese valor de r minimo
                xp=[xp[-1]] #me quedo con los últimos valores de t para que el programa pueda seguir funcionando pero que en la próxima vuelta no me imprima la q ya está impresa
                yp=[yp[-1]]
                rp=[]

                if vueltas>10: #tomamos 10 vueltas para evitar errores.
                    break

        else: #en este caso volvemos al punto de partida con otra h más pequeña
            h=h*rho**(1/4)
            r=copy(r0)
            r2=copy(r0)

    ang=[arctan(y0[0]/x0[0])+pi] #el ángulo q forma el primer mínimo del radio (pi con respecto al eje positivo de x)
    giro=[] #me voy a ir metiendo la diferencia de valores de ang por vuelta

    for w in range(int(vueltas)-1): #para sacar el ángulo de la posición de minima distancia con respecto al eje x positivo
        ang.append(arctan(y0[w+1]/x0[w+1]))
        if y0[w+1]>0 and x0[w+1]<0:
            ang[-1]+=pi
        if y0[w+1]<0 and x0[w+1]<0:
            ang[-1]+=pi
        if y0[w+1]<0 and x0[w+1]>0:
            ang[-1]+=2*pi
        giro.append((ang[w+1]-ang[w])/(t0[w+1]-t0[w])) #hago la diferencia de ángulos y divido entre la diferencia de tiempos (saldrá en radianes/año)

    print( "Giro [''/siglo]:",100*mean(giro)*180*3600/(pi*(63241/c)**2))#sacamos el valor del giro por siglo (pasamos de años a siglos (*100), de radianes a grados y despues a segundos (180*3600/pi) y corregimos el factor de c^-2  *1/(63241/c)**2)
    scatter(x0,y0) #dibujamos los puntos en los q se encunetra el semieje menor para cada vuelta
    scatter(0,0,c="yellow",label="Sol")
    title(valores[i][0])
    legend()
    xlabel("x")
    ylabel("y")
    ylim(-0.05,0.4)
    show() 

En la gráfica podemos ver como la órbita no es cerrada (parte superior de la imagen), a su vez, podemos ver como el punto donde se encuentra en el semieje menor va cambiando para cada vuelta pues los puntos naranjas se van moviendo. (Solo se ha dibujado la parte superior de la órbita para que se pudiese ver con claridad que la órbita no era cerrada)